- TODO: Parallelize sharp wave property computation

# Imports and definitions

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import json
from datetime import datetime

In [3]:
import ecephys_analyses as ea
from ecephys_analyses.load import load_sr_chans
from sglxarray import load_trigger
from ecephys.xrsig import get_kcsd
from ecephys.signal.sharp_wave_ripples import detect_sharp_waves_by_value, get_durations, get_midpoints, get_sink_amplitudes, get_sink_integrals
from ecephys.utils import store_df_h5

## Detect SPWS, accounting for drift

In [4]:
def get_epoch_spws(csd, spw_params, epoch_start, epoch_end, sr_chans):
    if not sr_chans:
        return pd.DataFrame()
    
    sr_csd = csd.sel(time=slice(epoch_start, epoch_end), channel=sr_chans)
    
    spws = detect_sharp_waves_by_value(
        sr_csd.time.values,
        sr_csd.values,
        spw_params["detection_threshold"],
        spw_params["boundary_threshold"],
        spw_params["minimum_duration"],
    )

    if not spws.empty:
        spws["duration"] = get_durations(spws)
        spws["midpoint"] = get_midpoints(spws)
        spws["sink_amplitude"] = get_sink_amplitudes(spws, sr_csd.time.values, sr_csd.values) * (
            1e-6
        )  # Scale to mA/mm
        spws["sink_integral"] = (
            get_sink_integrals(spws, sr_csd.time.values, sr_csd.fs, sr_csd.values) * (1e-6) * (1e3)
        )  # Scale to mA * ms

        spws["sr_chans"] = [sr_chans] * len(spws)
    
    return spws

In [5]:
def get_file_spws(bin_path, sr_chans_path, spw_path, params_path):
    with open(params_path) as params_file:
        spw_params = json.load(params_file)
        csd_params = spw_params["csd_params"]

    sig = load_trigger(bin_path, csd_params['csd_channels'])
        
    csd = get_kcsd(
        sig,
        np.arange(0, len(sig.channel)) * csd_params['electrode_pitch'],
        drop_chans=csd_params["channels_omitted_from_csd_estimation"],
        do_lcurve=False,
        gdx=csd_params["gdx"],
        R_init=csd_params["R"],
        lambd=csd_params["lambd"],
    ).swap_dims({'pos': 'channel'})

    sr_chans_df = load_sr_chans(sr_chans_path)
    spws_by_epoch = list()
    for epoch in sr_chans_df.itertuples():
        spws_by_epoch.append(
            get_epoch_spws(
                csd, spw_params, epoch.start_time, epoch.end_time, epoch.sr_chans
            )
        )

    spws = pd.concat(spws_by_epoch)
    metadata = dict(
        spw_params=spw_params,
        csd_params=csd_params,
        file_start=str(sig.datetime.values.min()),
    )
    metadata.update(spws.attrs)

    store_df_h5(spw_path, spws, **metadata)

In [6]:
def get_alias_spws(subject, experiment, alias, probe):
    bin_paths = ea.get_lfp_bin_paths(subject, experiment, alias, probe=probe)
    sr_chans_paths = ea.get_project_counterparts("SPWRs", subject, bin_paths, ".sr_chans.csv")
    spw_paths = ea.get_project_counterparts("SPWRs", subject, bin_paths, ".spws.h5")
    params_path = ea.get_experiment_subject_file("SPWRs", experiment, subject, "sharp_wave_detection_params.json")

    for bin_path, sr_chans_path, spw_path in zip(bin_paths, sr_chans_paths, spw_paths):
        get_file_spws(bin_path, sr_chans_path, spw_path, params_path)
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"{current_time}: Finished {str(bin_path)}")

# Run automated pipeline

In [7]:
get_alias_spws("CNPIX12-Santiago", "conveyor_over_water", "full_48h", "imec1")

nChan: 385, nFileSamp: 18000039


/tmp/ipykernel_955963/248070820.py:8: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['sr_chans'], dtype='object')]

  get_file_spws(bin_path, sr_chans_path, spw_path, params_path)


19:26:57: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX12-Santiago/11-3-2021/SpikeGLX/11-3-2021_g0/11-3-2021_g0_imec1/11-3-2021_g0_t0.imec1.lf.bin
nChan: 385, nFileSamp: 18000038
19:38:56: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX12-Santiago/11-3-2021/SpikeGLX/11-3-2021_g0/11-3-2021_g0_imec1/11-3-2021_g0_t1.imec1.lf.bin
nChan: 385, nFileSamp: 18000039
19:50:23: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX12-Santiago/11-3-2021/SpikeGLX/11-3-2021_g0/11-3-2021_g0_imec1/11-3-2021_g0_t2.imec1.lf.bin
nChan: 385, nFileSamp: 18000038
20:01:33: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX12-Santiago/11-3-2021/SpikeGLX/11-3-2021_g0/11-3-2021_g0_imec1/11-3-2021_g0_t3.imec1.lf.bin
nChan: 385, nFileSamp: 18000039
20:15:12: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX12-Santiago/11-3-2021/SpikeGLX/11-3-2021_g0/11-3-2021_g0_imec1/11-3-2021_g0_t4.imec1.lf.bin
nChan: 385, nFileSamp: 18000038
20:19:51: Finished /Volumes/neuropixel_archive/Data/chr